In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import gc
import random

# GPU 메모리 비우기 함수
def clear_gpu_memory():
    print("Clearing GPU memory...")
    torch.cuda.empty_cache()
    gc.collect()

# 예시: 학습 후 GPU 메모리 비우기
# 모델 학습 코드 (생략)
# ...

# 학습 후 GPU 메모리 비우기
clear_gpu_memory()

# 추가적으로 모델을 삭제하고 가비지 컬렉션 실행
#del model
gc.collect()


# 시드 설정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



# 데이터 경로 설정
data_csv_path = 'cv_challenge/train_augmented_byteam_rf_and_original.csv'
image_folder_path = 'cv_challenge/data_argumentation_byteam_rotate_flipped'
test_image_folder_path = 'cv_challenge/test'
output_image_folder_path = 'cv_challenge/denoised_images'

# 데이터 로드
df = pd.read_csv(data_csv_path)

Clearing GPU memory...


In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None, noise_factor=0.5):
        self.dataframe = dataframe
        self.image_folder = image_folder
        self.transform = transform
        self.noise_factor = noise_factor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        image = image.resize((224, 224))
        
        if self.transform:
            image = self.transform(image)
        
        noisy_image = image + self.noise_factor * torch.randn_like(image)
        noisy_image = torch.clamp(noisy_image, 0., 1.)
        
        return noisy_image, image

# 데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = CustomDataset(df, image_folder_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),  # 3x224x224 -> 32x112x112
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # 32x112x112 -> 64x56x56
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # 64x56x56 -> 128x28x28
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # 128x28x28 -> 256x14x14
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # 256x14x14 -> 128x28x28
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # 128x28x28 -> 64x56x56
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),  # 64x56x56 -> 32x112x112
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # 32x112x112 -> 3x224x224
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# 모델 초기화
autoencoder = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [6]:
num_epochs = 5

for epoch in range(num_epochs):
    for data in dataloader:
        noisy_img, clean_img = data
        noisy_img = noisy_img.cuda()
        clean_img = clean_img.cuda()
        
        # 순전파
        output = autoencoder(noisy_img)
        loss = criterion(output, clean_img)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5], Loss: 0.0058
Epoch [2/5], Loss: 0.0048
Epoch [3/5], Loss: 0.0052
Epoch [4/5], Loss: 0.0038
Epoch [5/5], Loss: 0.0036


In [7]:
class TestDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_files = os.listdir(image_folder)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path).convert('RGB')
        image = image.resize((224, 224))
        
        if self.transform:
            image = self.transform(image)
        
        return image, img_name

# 테스트 데이터 로드
test_dataset = TestDataset(test_image_folder_path, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 디노이즈된 이미지 저장 폴더 생성
if not os.path.exists(output_image_folder_path):
    os.makedirs(output_image_folder_path)

denoised_image_info = []

# 테스트 데이터 디노이즈 및 저장
with torch.no_grad():
    for data in test_dataloader:
        images, img_names = data
        images = images.cuda()
        
        outputs = autoencoder(images)
        outputs = outputs.cpu()

        for i in range(outputs.size(0)):
            img = outputs[i].numpy().transpose(1, 2, 0)  # 채널 순서 변경
            img = (img * 255).astype(np.uint8)
            img = Image.fromarray(img)
            
            output_img_name = f"denoised_{img_names[i]}"
            output_img_path = os.path.join(output_image_folder_path, output_img_name)
            img.save(output_img_path)
            
            denoised_image_info.append({
                'ID': output_img_name,
                'Original': img_names[i]
            })

# 디노이즈된 이미지 정보 저장
output_csv_path = 'cv_challenge/denoised_images_info.csv'
denoised_df = pd.DataFrame(denoised_image_info)
denoised_df.to_csv(output_csv_path, index=False)